In [1]:
import pandas as pd
from geopy.geocoders import Nominatim
import time
import sys
import os

In [ ]:
# Definición de rutas para importar y exportar datos repocitorio local

# Definición de rutas para importar y exportar datos repocitorio local // "EN ESTE ESPACIO DEBE IR LA RUTA DONDE DESEA GUARDAR SU PROYECTO."
raiz = r'RUTA DE SU PROYECTO\\Proyecto_ETL_Fuga_de_Cerebros' #<==================DEBES COLOCAR TU RUTA AQUI Pero siempre respetar el \\Proyecto_ETL_Fuga_de_Cerebros'

paths = {
     'path_import': f"{raiz}\\data\\import",
     'path_export': f"{raiz}\\data\\export",

}

print('Paths configurados correctamente')


Paths configurados correctamente


In [ ]:
# === CONFIGURACIÓN DE RUTAS y llamado de data ===

#Configuración de rutas de archivos
municipios_import = os.path.join(paths['path_import'], 'validacion_municipios.xlsx')
municipios_export = os.path.join(paths['path_export'], 'Ciudades_colombianas_con_coordenadas.xlsx')
# Nombre del archivo de entrada
archivo_entrada = municipios_import
archivo_salida = municipios_export

# Nombre de la hoja que contiene los datos
hoja = "ciudades_paises"

# === CARGAR DATOS ===
df = pd.read_excel(archivo_entrada, sheet_name=hoja)

print("Archivo leído correctamente ✅")
print("Filas:", df.shape[0])
print("Columnas:", df.shape[1])
print(df.head())

Archivo leído correctamente ✅
Filas: 421
Columnas: 5
  Pais de Nacimiento Departamento/Estado.1 Ciudad_Origen  Latitud  Longitud
0           COLOMBIA    NORTE DE SANTANDER     AGUACLARA      NaN       NaN
1           COLOMBIA                BOYACA       AGUAZUL      NaN       NaN
2           COLOMBIA       VALLE DEL CAUCA         ALBAN      NaN       NaN
3           COLOMBIA             ANTIOQUIA      ALTAMIRA      NaN       NaN
4           COLOMBIA                NARIÑO      ALTAMIRA      NaN       NaN


In [5]:
#========== Inicializar geocodificador ==========
geolocator = Nominatim(user_agent="ciudades_colombia_locator")

def obtener_coordenadas(ciudad, departamento):
    """Busca coordenadas de una ciudad en Colombia"""
    try:
        location = geolocator.geocode(f"{ciudad}, {departamento}, Colombia", timeout=15)
        if location:
            return location.latitude, location.longitude
    except Exception as e:
        print(f"Error con {ciudad}, {departamento}: {e}")
        return None, None
    return None, None

# Crear nuevas columnas
df["Latitud"] = None
df["Longitud"] = None

# === PROCESO DE GEOCODIFICACIÓN ===
for i, row in df.iterrows():
    ciudad = row["Ciudad_Origen"]
    depto = row["Departamento/Estado.1"]

    lat, lon = obtener_coordenadas(ciudad, depto)
    df.at[i, "Latitud"] = lat
    df.at[i, "Longitud"] = lon

    print(f"{i+1}/{len(df)} - {ciudad}, {depto} -> {lat}, {lon}")
    
    time.sleep(1.5)  # pausa para no saturar el servidor

# === GUARDAR RESULTADOS ===
df.to_excel(archivo_salida, index=False)
print(f"Archivo generado: {archivo_salida}")

#_________________________________________________________________________________________

1/421 - AGUACLARA, NORTE DE SANTANDER -> None, None
2/421 - AGUAZUL, BOYACA -> None, None
3/421 - ALBAN, VALLE DEL CAUCA -> 4.7835097, -76.174142
4/421 - ALTAMIRA, ANTIOQUIA -> 6.234076, -75.977152
5/421 - ALTAMIRA, NARIÑO -> 1.1991961, -77.296115


KeyboardInterrupt: 